<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML321ENSkillsNetwork817-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Extract Bag of Words (BoW) Features from Course Textual Content**


Estimated time needed: **60** minutes


The main goal of recommender systems is to help users find items they potentially interested in. Depending on the recommendation tasks, an item can be a movie, a restaurant, or, in our case, an online course. 

Machine learning algorithms cannot work on an item directly so we first need to extract features and represent the items mathematically, i.e., with a feature vector.

Many items are often described by text so they are associated with textual data, such as the titles and descriptions of a movie or course. Since machine learning algorithms can not process textual data directly, we need to transform the raw text into numeric feature vectors.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/module_2/images/extract_textual_features.png)


In this lab, you will be learning to extract the bag of words (BoW) features from course titles and descriptions. The BoW feature is a simple but effective feature characterizing textual data and is widely used in many textual machine learning tasks.


## Objectives


After completing this lab you will be able to:


* Extract Bag of Words (BoW) features from course titles and descriptions
* Build a course BoW dataset to be used for building a content-based recommender system later


----


## Prepare and setup the lab environment


First, let's install and import required libraries:


In [1]:
!pip install nltk==3.6.7
!pip install gensim
!pip install scipy==1.10
!pip install pandas
!pip install nltk
!pip install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 186.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 158.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 71.8 MB/s eta 0:00:00:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: Ignored the following yanked versions: 1.11.0, 1.14.0rc1
ERROR: Could not find a version that satisfies the requirement scipy==1.10 (from versions: 0.8.0, 0.9.0, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 0.13.2, 0.13.3, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.16.0, 0.16.1, 0.17.0, 0.17.1, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.3.0, 1.3.1, 1.3.2, 1.3.3, 1.4.0, 1.4.1, 1.5.0, 1.5.1, 1.5.2, 

In [2]:
import gensim
import pandas as pd
import nltk as nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora

%matplotlib inline

Download stopwords


In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyterlab/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
# also set a random state
rs = 123

### Bag of Words (BoW) features


BoW features are essentially the counts or frequencies of each word that appears in a text (string). Let's illustrate it with some simple examples.


Suppose we have two course descriptions as follows:


In [7]:
course1 = "this is an introduction data science course which introduces data science to beginners"

In [5]:
course2 = "machine learning for beginners"

In [8]:
courses = [course1, course2]
courses

['this is an introduction data science course which introduces data science to beginners',
 'machine learning for beginners']

The first step is to split the two strings into words (tokens). A token in the text processing context means the smallest unit of text such as a word, a symbol/punctuation, or a phrase, etc. The process to transform a string into a collection of tokens is called `tokenization`.


One common way to do ```tokenization``` is to use the Python built-in `split()` method of the `str` class.  However, in this lab, we want to leverage the `nltk` (Natural Language Toolkit) package, which is probably the most commonly used package to process text or natural language.


 More specifically, we will use the ```word_tokenize()``` method on the content of course (string):


In [9]:
# Tokenize the two courses
tokenized_courses = [word_tokenize(course) for course in courses]

In [10]:
tokenized_courses

[['this',
  'is',
  'an',
  'introduction',
  'data',
  'science',
  'course',
  'which',
  'introduces',
  'data',
  'science',
  'to',
  'beginners'],
 ['machine', 'learning', 'for', 'beginners']]

As you can see from the cell output, two courses have been tokenized and turned into two token arrays.


Next, we want to create a token dictionary to index all tokens. Basically, we want to assign a key/index for each token. One way to index tokens is to use the `gensim` package which is another popular package for processing textual data:


In [11]:
# Create a token dictionary for the two courses
tokens_dict = gensim.corpora.Dictionary(tokenized_courses)

In [12]:
print(tokens_dict.token2id)

{'an': 0, 'beginners': 1, 'course': 2, 'data': 3, 'introduces': 4, 'introduction': 5, 'is': 6, 'science': 7, 'this': 8, 'to': 9, 'which': 10, 'for': 11, 'learning': 12, 'machine': 13}


With the token dictionary, we can easily count each token in the two example courses and output two BoW feature vectors. However, more conveniently, the `gensim` package provides us a `doc2bow` method to generate BoW features out-of-box.


In [13]:
# Generate BoW features for each course
courses_bow = [tokens_dict.doc2bow(course) for course in tokenized_courses]

In [14]:
courses_bow

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 2),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 1),
  (10, 1)],
 [(1, 1), (11, 1), (12, 1), (13, 1)]]

It outputs two BoW arrays where each element is a tuple, e.g., (0, 1) and (7, 2). The first element of the tuple is the token ID and the second element is its count. So `(0, 1)` means `(``an``, 1)` and `(7, 2)` means `(``science``, 2)`.


We can use the following code snippet to print each token and its count:


In [15]:
# Enumerate through each course and its bag-of-words representation
for course_idx, course_bow in enumerate(courses_bow):
    # Print the index of the current course and a label
    print(f"Bag of words for course {course_idx}:")
    # For each token index, print its bow value (word count)
    for token_index, token_bow in course_bow:
        # Retrieve the token from the tokens dictionary based on its index
        token = tokens_dict.get(token_index)
        # Print the token and its bag-of-words value
        print(f"--Token: '{token}', Count:{token_bow}")

Bag of words for course 0:
--Token: 'an', Count:1
--Token: 'beginners', Count:1
--Token: 'course', Count:1
--Token: 'data', Count:2
--Token: 'introduces', Count:1
--Token: 'introduction', Count:1
--Token: 'is', Count:1
--Token: 'science', Count:2
--Token: 'this', Count:1
--Token: 'to', Count:1
--Token: 'which', Count:1
Bag of words for course 1:
--Token: 'beginners', Count:1
--Token: 'for', Count:1
--Token: 'learning', Count:1
--Token: 'machine', Count:1


If we turn to the long list into a horizontal feature vectors, we can see the two courses become two numerical feature vectors:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/module_2/images/bow.png)


### BoW dimensionality reduction


A document may contain tens of thousands of words which makes the dimension of the BoW feature vector huge. To reduce the dimensionality, one common way is to filter the relatively meaningless tokens such as stop words or sometimes add position and adjective words.


We can use the english stop words provided in `nltk`:


In [16]:
stop_words = set(stopwords.words('english'))

In [18]:
#stop_words

Then we can filter those English stop words from the tokens in course1:


In [19]:
# Tokens in course 1
tokenized_courses[0]

['this',
 'is',
 'an',
 'introduction',
 'data',
 'science',
 'course',
 'which',
 'introduces',
 'data',
 'science',
 'to',
 'beginners']

In [20]:
processed_tokens = [w for w in tokenized_courses[0] if not w.lower() in stop_words]

In [21]:
processed_tokens

['introduction',
 'data',
 'science',
 'course',
 'introduces',
 'data',
 'science',
 'beginners']

You can see the number of tokens for ```course1``` has been reduced.


Another common way is to only keep nouns in the text. We can use the `nltk.pos_tag()` method to analyze the part of speech (POS) and annotate each word.


In [22]:
tags = nltk.pos_tag(tokenized_courses[0])
tags

[('this', 'DT'),
 ('is', 'VBZ'),
 ('an', 'DT'),
 ('introduction', 'NN'),
 ('data', 'NNS'),
 ('science', 'NN'),
 ('course', 'NN'),
 ('which', 'WDT'),
 ('introduces', 'VBZ'),
 ('data', 'NNS'),
 ('science', 'NN'),
 ('to', 'TO'),
 ('beginners', 'NNS')]

As we can see [`introduction`, `data`, `science`, `course`, `beginners`] are all of the nouns and we may keep them in the BoW feature vector.


### TASK: Extract BoW features for course textual content and build a dataset


By now you have learned what a BoW feature is, so let's start extracting BoW features from some real course textual content.


In [23]:
course_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/datasets/course_processed.csv"
course_content_df = pd.read_csv(course_url)

In [25]:
course_content_df.head(2)

,COURSE_ID,TITLE,DESCRIPTION
0,ML0201EN,robots are coming build iot apps with watson ...,have fun with iot and learn along the way if ...
1,ML0122EN,accelerating deep learning with gpu,training complex deep learning models with lar...


The course content dataset has three columns `COURSE_ID`, `TITLE`, and `DESCRIPTION`. `TITLE` and `DESCRIPTION` are all text upon which we want to extract BoW features. 


Let's join those two text columns together.


In [26]:
# Merge TITLE and DESCRIPTION title
course_content_df['course_texts'] = course_content_df[['TITLE', 'DESCRIPTION']].agg(' '.join, axis=1)
course_content_df = course_content_df.reset_index()
course_content_df['index'] = course_content_df.index

In [27]:
course_content_df.head(2)

,index,COURSE_ID,TITLE,DESCRIPTION,course_texts
0,0,ML0201EN,robots are coming build iot apps with watson ...,have fun with iot and learn along the way if ...,robots are coming build iot apps with watson ...
1,1,ML0122EN,accelerating deep learning with gpu,training complex deep learning models with lar...,accelerating deep learning with gpu training c...


We have used the `tokenize_course()` method  to tokenize the course content:


In [28]:
def tokenize_course(course, keep_only_nouns=True):
    # Get English stop words
    stop_words = set(stopwords.words('english'))
    # Tokenize the course text
    word_tokens = word_tokenize(course)
    # Remove English stop words and numbers
    word_tokens = [w for w in word_tokens if (not w.lower() in stop_words) and (not w.isnumeric())]
    # Only keep nouns 
    if keep_only_nouns:
        # Define a filter list of non-noun POS tags
        filter_list = ['WDT', 'WP', 'WRB', 'FW', 'IN', 'JJR', 'JJS', 'MD', 'PDT', 'POS', 'PRP', 'RB', 'RBR', 'RBS',
                       'RP']
        # Tag the word tokens with POS tags
        tags = nltk.pos_tag(word_tokens)
        # Filter out non-nouns based on POS tags
        word_tokens = [word for word, pos in tags if pos not in filter_list]

    return word_tokens

Let's try it on the first course.


In [29]:
a_course = course_content_df.iloc[0, :]['course_texts']
a_course

'robots are coming  build iot apps with watson  swift  and node red have fun with iot and learn along the way  if you re a swift developer and want to learn more about iot and watson ai services in the cloud  raspberry pi   and node red  you ve found the right place  you ll build iot apps to read temperature data  take pictures with a raspcam  use ai to recognize the objects in those pictures  and program an irobot create 2 robot  '

In [30]:
tokenize_course(a_course)

['robots',
 'coming',
 'build',
 'iot',
 'apps',
 'watson',
 'swift',
 'red',
 'fun',
 'iot',
 'learn',
 'way',
 'swift',
 'developer',
 'want',
 'learn',
 'iot',
 'watson',
 'ai',
 'services',
 'cloud',
 'raspberry',
 'pi',
 'node',
 'red',
 'found',
 'place',
 'build',
 'iot',
 'apps',
 'read',
 'temperature',
 'data',
 'take',
 'pictures',
 'raspcam',
 'use',
 'ai',
 'recognize',
 'objects',
 'pictures',
 'program',
 'irobot',
 'create',
 'robot']

Next, you will need to write some code snippets to generate the BoW features for each course. Let's start by tokenzing all courses in the `courses_df`:


_TODO: Use provided tokenize_course() method to tokenize all courses in courses_df['course_texts']._


In [31]:
# WRITE YOUR CODE HERE
# Apply the tokenize_course function to each row in the 'course_texts' column
course_content_df['tokens'] = course_content_df['course_texts'].apply(tokenize_course)

# View the updated DataFrame
course_content_df.head()

,index,COURSE_ID,TITLE,DESCRIPTION,course_texts,tokens
0,0,ML0201EN,robots are coming build iot apps with watson ...,have fun with iot and learn along the way if ...,robots are coming build iot apps with watson ...,"[robots, coming, build, iot, apps, watson, swi..."
1,1,ML0122EN,accelerating deep learning with gpu,training complex deep learning models with lar...,accelerating deep learning with gpu training c...,"[accelerating, deep, learning, gpu, training, ..."
2,2,GPXX0ZG0EN,consuming restful services using the reactive ...,learn how to use a reactive jax rs client to a...,consuming restful services using the reactive ...,"[consuming, restful, services, using, reactive..."
3,3,RP0105EN,analyzing big data in r using apache spark,apache spark is a popular cluster computing fr...,analyzing big data in r using apache spark apa...,"[analyzing, big, data, r, using, apache, spark..."
4,4,GPXX0Z2PEN,containerizing packaging and running a sprin...,learn how to containerize package and run a ...,containerizing packaging and running a sprin...,"[containerizing, packaging, running, spring, b..."



<details>
    <summary>Click here for Hints</summary>

Use `tokenize_course(text, True)` command to tokenize each text in `courses_df['course_texts']`


Then we need to create a token dictionary `tokens_dict`


_TODO: Use gensim.corpora.Dictionary(tokenized_courses) to create a token dictionary._


In [32]:
# WRITE YOUR CODE HERE

# Get the list of tokenized courses
tokenized_courses = course_content_df['tokens'].tolist()

#Create a Gensim dictionary from the tokenized courses
token_dictionary = gensim.corpora.Dictionary(tokenized_courses)

Then we can use `doc2bow()` method to generate BoW features for each tokenized course.


_TODO: Use tokens_dict.doc2bow() to generate BoW features for each tokenized course._


In [33]:
# WRITE YOUR CODE HERE

#Create BoW corpus using the dictionary
bow_corpus = [token_dictionary.doc2bow(tokens) for tokens in tokenized_courses]

In [36]:
len(bow_corpus)

307

<details>
    <summary>Click here for Hints</summary>
    
You can use `tokens_dict.doc2bow(course)` command  for each course in `tokenized_courses`


Lastly, you need to append the BoW features for each course into a new BoW dataframe. The new dataframe needs to include the following columns (you may include other relevant columns as well):
- 'doc_index': the course index starting from 0
- 'doc_id': the actual course id such as `ML0201EN`
- 'token': the tokens for each course
- 'bow': the bow value for each token


_TODO: Create a new course_bow dataframe based on the extracted BoW features._


In [43]:
course_ids = course_content_df['COURSE_ID'].tolist()

# Prepare rows
rows = []
for idx, (course_id, bow) in enumerate(zip(course_ids, bow_corpus)):
    for token_id, count in bow:
        token = tokens_dict.id2token.get(token_id, f"[UNK_{token_id}]")
        rows.append({
            'doc_index': idx,
            'doc_id': course_id,
            'token': token,
            'bow': count
        })

# Convert to dataframe
bow_df = pd.DataFrame(rows)
bow_df.head()

,doc_index,doc_id,token,bow
0,0,ML0201EN,an,2
1,0,ML0201EN,beginners,2
2,0,ML0201EN,course,2
3,0,ML0201EN,data,1
4,0,ML0201EN,introduces,1


In [40]:
# WRITE YOUR CODE HERE
# Construct the BoW dataframe
# bow_df = pd.DataFrame([
#     {
#         'doc_index': idx,
#         'doc_id': course_content_df.loc[idx, 'COURSE_ID'],
#         'token': tokenized_courses[idx],
#         'bow': bow_corpus[idx]
#     }
#     for idx in range(len(tokenized_courses))
# ])

# bow_df.head()

#  ...
#  bow_dicts = {"doc_index": doc_indices,
#            "doc_id": doc_ids,
#            "token": tokens,
#            "bow": bow_values}
#  pd.DataFrame(bow_dicts)

,doc_index,doc_id,token,bow
0,0,ML0201EN,"[robots, coming, build, iot, apps, watson, swi...","[(0, 2), (1, 2), (2, 2), (3, 1), (4, 1), (5, 1..."
1,1,ML0122EN,"[accelerating, deep, learning, gpu, training, ...","[(0, 2), (3, 2), (6, 1), (12, 1), (30, 4), (34..."
2,2,GPXX0ZG0EN,"[consuming, restful, services, using, reactive...","[(12, 1), (26, 1), (30, 1), (108, 2), (109, 1)..."
3,3,RP0105EN,"[analyzing, big, data, r, using, apache, spark...","[(6, 4), (63, 1), (77, 1), (83, 1), (117, 1), ..."
4,4,GPXX0Z2PEN,"[containerizing, packaging, running, spring, b...","[(12, 1), (140, 2), (141, 2), (142, 1), (143, ..."


In [41]:
course_content_df.shape

(307, 6)

<details>
    <summary>Click here for Hints</summary>
    
You can use 2 for-loops to create your data frame: first one will be `for doc_index, doc_bow in enumerate(bow_docs):` where bow_docs is the list of BoW features for each tokenized course and within this for-loop you will have another loop `for token_index, token_bow in doc_bow:`. Then you can get each "token" by applying the `token_index` to your `token_dict`,  `token_bow` will give you "bow" values, `doc_indices` will give you values for  "doc_index" and you can get "doc_id" by using `courses_df['COURSE_ID']` list and `doc_index` as indexes.


Your course BoW dataframe may look like the following:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML321EN-SkillsNetwork/labs/module_2/images/bow_dataset.png)


You may refer to previous code examples in this lab if you need help with creating the BoW dataframe.


### Other popular textual features


In addition to the basic token BoW feature, there are two other types of widely used textual features. If you are interested, you may explore them yourself to learn how to extract them from the course textual content: 


- **tf-idf**: tf-idf refers to Term Frequency–Inverse Document Frequency. Similar to BoW, the tf-idf also counts the word frequencies in each document. Furthermore, tf-idf will  offset the number of documents in the corpus that contain the word in order to adjust for the fact that some words appear more frequently in general. The higher the tf-idf normally means the greater the importance the word/token is.
- **Text embedding vector**. Embedding means projecting an object into a latent feature space. We normally employ neural networks or deep neural networks to learn the latent features of a textual object such as a word, a sentence, or the entire document. The learned latent feature vectors will be used to represent the original textual entities. 


### Summary


Congratulations, you have completed the BoW feature extraction lab. In this lab, you have learned and practiced extracting BoW features from course titles and descriptions. Once the feature vectors on the courses has been built, we can then apply machine learning algorithms such as similarity measurements, clustering, or classification on the courses in later labs.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


```toggle## Change Log
```


```toggle|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
```
```toggle|-|-|-|-|
```
```toggle|2021-10-25|1.0|Yan|Created the initial version|
```


Copyright © 2021 IBM Corporation. All rights reserved.
